<a href="https://colab.research.google.com/github/aradhyxsingh/ChatBot_UsingNN/blob/master/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##***Ever wanted to create an AI Chat bot? This python chatbot project will show you how to create a chatbot with python using deep learning.*** 

###Steps to run this:

1.   Click on 1st checkpoint then press shift+enter this will take you to the 2nd cell hit shift+enter again you'll arive at the 2nd checkpoint.
2.   Press shift+enter again after the cell executes it'll provide you a link in blue colour.After opening that link:

        *   Choose your google account and hit allow.
        *   You'll be given a verification code, copy that and paste it in the input box and press enter
3.   This is your 3rd Checkpoint just press shift+enter and wait till your chatbot finishes it's training, after execution you'll be at you 4th and final checkpoint
4.   Press shift+enter again, answer a small question and you're good to go.Enjoy







In [2]:
pip install tflearn #1st Checkpoint

     |████████████████████████████████| 102kB 3.3MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128207 sha256=e2149df816e49d25d586cdb06453eaeb01d0956318a28002918a3f6f17ed30ab
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [18]:
pip install tensorflow==1.15

In [5]:
#2nd Checkpoint
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import random
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import random
import tflearn
import io
import json
from multiprocessing import Process, Manager
import pickle
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id ='1XuVUIM2-DWA5IgxoSllJ7gx3kNHF6iV3'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('chatbot.json')
data=pd.read_json('chatbot.json')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
#3rd Checkpoint
words =[]
labels = []
docs_x =[]
docs_y = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
      labels.append(intent['tag'])

words = [ stemmer.stem(w.lower()) for w in words if w != '?']

words = sorted(list(set(words)))

labels = sorted(labels)

training =[]
output = []

out_empty = [ 0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w) for w in doc]
  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)
  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1
  training.append(bag)
  output.append(output_row)
training = np.array(training)
output = np.array(output)

from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]),activation='softmax')
net = tflearn.regression(net)
model = tflearn.DNN(net)

model.fit(training,output, n_epoch=1000,batch_size=8,show_metric=True)
model.save("model.h5")

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

Training Step: 3999  | total loss: 0.00463 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.00463 - acc: 0.9998 -- iter: 24/27
Training Step: 4000  | total loss: 0.00437 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.00437 - acc: 0.9999 -- iter: 27/27
--
INFO:tensorflow:/content/model.h5 is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
#4th Checkpoint
def chat():
    while True:
      x=input("Your Question: ")
      if x=='youranswer':
        print(" Congratulationssss here's your BOT(type quit to go back to the question)!")
        while True:
            inp = input("You: ")
            if inp.lower() == "quit":
                break


            results = model.predict([bag_of_words(inp, words)])[0]
            results_index = np.argmax(results)
            tag = labels[results_index]

            if results[results_index] > 0.73:
              for tg in data["intents"]:
                if tg['tag'] == tag:
                  responses = tg['responses']
              print("Bot:",random.choice(responses))
            else:
              print("I did't get this but there's a lot to explore, i like challenges though xD")
      elif x.lower()=='quit':       
        break
      else:
        print('Wrong answer try again')
      
chat()	
#It answers your questions even if they're wrongly spelt due to the use of Neural Networks

Your Question: youranswer
 Congratulationssss here's your BOT(type quit to go back to the question)!
You: Is anyone there?
Bot: Good to see you again!
You: Is anyone the?
Bot: Hello!
You: Is anyonw ther?
Bot: Hello!
You: what is your age?
Bot: 21 years young!
You: how old arw you?
Bot: I am 21 years old!
You: some random irrelevant question ?
I did't get this but there's a lot to explore, i like challenges though xD
You: what can you do?
Bot: Ask me a question and I'll see if i can help you
You: what is your favourite colour?
Bot: I looooook amazing in my favorite colour that is RED
You: see you later!!
Bot: Talk to you later
You: bye
Bot: Talk to you later
You: quit
Your Question: quit
